<a href="https://colab.research.google.com/github/baranceanuvlad/Advanced-Topics-in-Neural-Networks-Template-2023/blob/main/Lab02/Homework2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorboard
import math
import random

In [55]:
x = torch.rand(20, 784).to('cuda')
W = torch.rand(784,10).to('cuda')
b = torch.rand(1, 10).to('cuda')
learning_rate = 0.1
y = torch.zeros((20, 10)).to('cuda')

for result in y:
  nr = random.randint(0, 9)
  result[nr] = 1

In [109]:
def sigmoid_function(z):
  return 1 / (1 + torch.exp(-z))

In [88]:
def train_perceptron(x, W, b, learning_rate, y_true):
  for epoch in range(1000):
    predicted_y = x @ W + b
    error = y_true - sigmoid_function(predicted_y)
    W = W + learning_rate * x.T @ error
    b = b + learning_rate * error.mean(axis = 0)
  return W, b

In [56]:
train_perceptron(x, W, b, learning_rate, y)

(tensor([[2.0758e-01, 1.3427e+02, 2.5051e+01,  ..., 4.0698e+01, 4.4462e+00,
          1.5955e+02],
         [9.6366e-01, 8.1246e+01, 8.3500e+00,  ..., 8.4931e+01, 4.3677e+01,
          1.3453e+02],
         [5.4827e-01, 2.2507e+02, 5.1627e+01,  ..., 1.6124e+02, 9.5314e+01,
          1.2673e+02],
         ...,
         [2.8894e-01, 1.2814e+02, 1.3641e+01,  ..., 7.9906e+01, 7.2053e+01,
          1.4220e+02],
         [8.4885e-01, 1.0923e+02, 8.8877e+01,  ..., 9.3570e+01, 1.7941e+01,
          1.3453e+02],
         [4.4830e-01, 1.2080e+02, 1.5629e+01,  ..., 2.4587e+01, 8.9984e+01,
          8.1109e+01]], device='cuda:0'),
 tensor([[ 0.1181, 15.5435,  5.7533, 15.1530,  5.8317, 10.0061, 20.7145, 10.8353,
           5.0155, 15.4474]], device='cuda:0'))

Bonus


In [110]:
train = pd.read_csv('mnist_train.csv')
labels = train['label']

train_tensor = torch.tensor(train.drop( 'label', axis = 1).values, dtype=torch.double).to('cuda')
train_tensor = (train_tensor - 127.5) / 127.5
#train_tensor = torch.nn.functional.normalize(train_tensor)
print(train_tensor)
y = torch.zeros((len(labels), 10)).to('cuda')
for idx, label in enumerate(labels):
  y[idx][label] = 1
b = torch.rand(1, 10).to('cuda')
learning_rate = 0.10
print(W)
W = torch.rand((784,10), dtype=torch.double).to('cuda')
print(W)

tensor([[-1., -1., -1.,  ..., -1., -1., -1.],
        [-1., -1., -1.,  ..., -1., -1., -1.],
        [-1., -1., -1.,  ..., -1., -1., -1.],
        ...,
        [-1., -1., -1.,  ..., -1., -1., -1.],
        [-1., -1., -1.,  ..., -1., -1., -1.],
        [-1., -1., -1.,  ..., -1., -1., -1.]], device='cuda:0',
       dtype=torch.float64)
tensor([[0.2966, 0.5562, 0.4337,  ..., 0.8104, 0.2731, 0.9187],
        [0.6856, 0.2547, 0.4254,  ..., 0.4453, 0.4936, 0.9526],
        [0.2436, 0.6034, 0.8899,  ..., 0.1466, 0.9940, 0.7011],
        ...,
        [0.4207, 0.9117, 0.9151,  ..., 0.8130, 0.8581, 0.0836],
        [0.9898, 0.8427, 0.5391,  ..., 0.8732, 0.1249, 0.4779],
        [0.5285, 0.2654, 0.7939,  ..., 0.9106, 0.6353, 0.0192]],
       device='cuda:0', dtype=torch.float64)
tensor([[0.0657, 0.4283, 0.6903,  ..., 0.7439, 0.3673, 0.6713],
        [0.8784, 0.4626, 0.3434,  ..., 0.1699, 0.9583, 0.5748],
        [0.0488, 0.5259, 0.9795,  ..., 0.2530, 0.2133, 0.3691],
        ...,
        [0.5644, 

In [111]:
test = pd.read_csv('mnist_test.csv')
labels = test['label']
test_tensor = torch.tensor(test.drop( 'label', axis = 1).values,dtype=torch.double).to('cuda')
test_tensor = (test_tensor - 127.5) / 127.5
y_test = torch.zeros((len(labels), 10)).to('cuda')
for idx, label in enumerate(labels):
  y_test[idx][label] = 1

In [112]:
count = 0
predicted_y = sigmoid_function(test_tensor @ W + b)
print(predicted_y)
for idx, line in enumerate(y_test):
  if torch.argmax(line) == torch.argmax(predicted_y[idx]):
    count += 1

print(W)
print(count * 100 / len(test_tensor))
print(count, len(test_tensor))

tensor([[5.4990e-141, 5.2747e-136, 1.5874e-136,  ..., 1.2641e-137,
         3.8071e-148, 2.2989e-135],
        [9.1120e-122, 2.1832e-120, 9.3851e-120,  ..., 2.3132e-127,
         4.1691e-127, 2.9685e-118],
        [8.3041e-151, 1.0506e-148, 9.4135e-151,  ..., 3.3017e-154,
         2.2882e-158, 3.0247e-152],
        ...,
        [1.0163e-122, 1.7221e-115, 7.2600e-116,  ..., 1.1619e-120,
         3.5367e-129, 9.0215e-118],
        [6.2961e-124, 3.5520e-122, 3.0718e-123,  ..., 3.4451e-128,
         3.0238e-131, 7.8119e-121],
        [4.0806e-103,  1.2632e-97,  1.4605e-93,  ..., 3.1668e-100,
          2.4307e-98,  5.2731e-98]], device='cuda:0', dtype=torch.float64)
tensor([[0.0657, 0.4283, 0.6903,  ..., 0.7439, 0.3673, 0.6713],
        [0.8784, 0.4626, 0.3434,  ..., 0.1699, 0.9583, 0.5748],
        [0.0488, 0.5259, 0.9795,  ..., 0.2530, 0.2133, 0.3691],
        ...,
        [0.5644, 0.9053, 0.4973,  ..., 0.9987, 0.4981, 0.4902],
        [0.8625, 0.4075, 0.3812,  ..., 0.0125, 0.1774, 0.5534

In [113]:

W, b = train_perceptron(train_tensor, W, b, learning_rate, y)

In [114]:



count = 0
predicted_y = sigmoid_function(test_tensor @ W + b)
predicted_y = torch.argmax(predicted_y, dim = 1)
for idx, line in enumerate(y_test):
  if torch.argmax(line) == predicted_y[idx]:
    count += 1

print(W)
print(count * 100 / len(test_tensor))
print(count, len(test_tensor))

tensor([[ 623.9806, 1145.8841,  501.5939,  ...,  763.9799,  835.3563,
         1644.9595],
        [ 624.7933, 1145.9185,  501.2470,  ...,  763.4058,  835.9473,
         1644.8629],
        [ 623.9637, 1145.9817,  501.8831,  ...,  763.4890,  835.2023,
         1644.6572],
        ...,
        [ 624.4792, 1146.3611,  501.4009,  ...,  764.2347,  835.4871,
         1644.7784],
        [ 624.7774, 1145.8633,  501.2848,  ...,  763.2485,  835.1664,
         1644.8416],
        [ 624.5886, 1146.3488,  501.5378,  ...,  763.9177,  835.5550,
         1645.0215]], device='cuda:0', dtype=torch.float64)
86.48
8648 10000
